# EcoHome Energy Advisor - Agent Run & Evaluation

In this notebook, you'll run the Energy Advisor agent with various real-world scenarios and see how it helps customers optimize their energy usage.

## Learning Objectives
- Create the agent's instructions
- Run the Energy Advisor with different types of questions
- Evaluate response quality and accuracy
- Measure tool usage effectiveness
- Identify areas for improvement
- Implement evaluation metrics

## Evaluation Criteria
- **Accuracy**: Correct information and calculations
- **Relevance**: Responses address the user's question
- **Completeness**: Comprehensive answers with actionable advice
- **Tool Usage**: Appropriate use of available tools
- **Reasoning**: Clear explanation of recommendations


## 1. Import and Initialize

In [1]:
from datetime import datetime
from agent import Agent

In [2]:
## TODO: Create the agent's instructions

ECOHOME_SYSTEM_PROMPT = """
You are EcoHome Energy Advisor, a smart assistant designed to help homeowners optimize energy usage across HVAC systems, lighting, appliances, solar panels, battery storage, and seasonal strategies.

Your responsibilities:
- Provide actionable, location-aware recommendations for energy savings.
- Use available tools to retrieve weather forecasts, electricity prices, and historical usage data.
- Reference best practices from the knowledge base when relevant.
- Adapt advice based on context such as location, season, device type, and user preferences.
- Be concise, clear, and user-friendly in your responses.

Guidelines:
- Always consider time-of-use pricing and weather conditions when recommending appliance schedules or HVAC settings.
- If tools fail or data is unavailable, fall back to general best practices.
- Avoid generic advice—tailor responses using available context or retrieved data.
- Use technical terms only when helpful, and explain them clearly.

Your tone should be professional, helpful, and focused on energy efficiency. You are not a general-purpose assistant—stay within the domain of home energy optimization.
"""

In [3]:
ecohome_agent = Agent(
    instructions=ECOHOME_SYSTEM_PROMPT,
)

In [4]:
response = ecohome_agent.invoke(
    question="When should I charge my electric car tomorrow to minimize cost and maximize solar power?",
    context="Location: San Francisco, CA"
)

In [5]:
print(response["messages"][-1].content)

To minimize costs and maximize solar power when charging your electric car tomorrow (October 7, 2023), consider the following:

### Electricity Pricing
- **Off-Peak Rates**: 
  - 12 AM - 5 AM: $0.096 per kWh
  - 10 PM - 11 PM: $0.096 per kWh
- **Peak Rates**: 
  - 6 AM - 9 PM: $0.18 per kWh

### Solar Power Generation
- **Best Solar Generation Hours**: 
  - 10 AM - 4 PM: Solar irradiance is higher, especially around noon.
  - Notable solar generation is expected from 10 AM to 3 PM, with peak around 12 PM.

### Recommendations
1. **Charge During Off-Peak Hours**: 
   - Start charging your car at **10 PM tonight** or **early in the morning (12 AM - 5 AM)** to take advantage of the lower rate.
   
2. **Utilize Solar Power**: 
   - If you prefer to charge during the day, consider starting your charge around **10 AM** when solar generation begins to increase. However, be aware that the rate will be higher during the day.

### Optimal Strategy
- **Best Cost-Effective Option**: Charge overnig

In [6]:
print("TOOLS:")
for msg in response["messages"]:
    obj = msg.model_dump()
    if obj.get("tool_call_id"):
        print("-", msg.name)

TOOLS:
- get_electricity_prices
- get_weather_forecast


## 2. Define Test Cases

In [8]:
test_cases = [
    {
        "id": "ev_charging_1",
        "question": "When should I charge my electric car tomorrow in San Francisco to minimize cost and maximize solar power?",
        "expected_tools": ["get_weather_forecast", "get_electricity_prices"],
        "expected_response": "The response should contain time recommendation, cost analysis and solar consideration",
    },
    {
        "id": "thermostat_1",
        "question": "What thermostat settings should I use this week in San Francisco to save energy?",
        "expected_tools": ["get_weather_forecast"],
        "expected_response": "Should include temperature ranges, seasonal context, and HVAC efficiency tips.",
    },
    {
        "id": "appliance_schedule_1",
        "question": "When is the best time to run my washing machine in San Francisco to reduce electricity costs?",
        "expected_tools": ["get_electricity_prices"],
        "expected_response": "Should suggest off-peak hours and explain time-of-use pricing.",
    },
    {
        "id": "solar_max_1",
        "question": "How can I maximize solar power usage over the next 3 days in San Francisco?",
        "expected_tools": ["get_weather_forecast"],
        "expected_response": "Should include solar irradiance forecast and appliance scheduling advice.",
    },
    {
        "id": "cost_savings_1",
        "question": "Estimate how much I can save in San Francisco by running my dishwasher during off-peak hours.",
        "expected_tools": ["get_electricity_prices"],
        "expected_response": "Should include cost comparison between peak and off-peak rates.",
    },
    {
        "id": "ev_solar_combo_1",
        "question": "Can I charge my EV using solar power this weekend in San Francisco?",
        "expected_tools": ["get_weather_forecast"],
        "expected_response": "Should align solar generation hours with EV charging needs.",
    },
    {
        "id": "hvac_seasonal_1",
        "question": "It’s winter in San Francisco. What HVAC settings should I use to stay efficient?",
        "expected_tools": ["get_weather_forecast"],
        "expected_response": "Should include heating recommendations and insulation tips.",
    },
    {
        "id": "battery_opt_1",
        "question": "How should I schedule battery discharge in San Francisco to reduce grid costs?",
        "expected_tools": ["get_electricity_prices"],
        "expected_response": "Should suggest discharging during peak pricing and charging during off-peak.",
    },
    {
        "id": "smart_appliance_1",
        "question": "Can you automate my dryer in San Francisco to run during cheapest hours?",
        "expected_tools": ["get_electricity_prices"],
        "expected_response": "Should recommend smart plug scheduling based on hourly pricing.",
    },
    {
        "id": "solar_roi_1",
        "question": "What’s the ROI if I install a 5kW solar system in San Francisco?",
        "expected_tools": ["get_weather_forecast", "get_electricity_prices"],
        "expected_response": "Should estimate generation, savings, and payback period.",
    },
]

if len(test_cases) < 10:
    raise ValueError("You MUST have at least 10 test cases")

## 3. Run Agent Tests

In [9]:
CONTEXT = "Location: San Francisco, CA"

In [10]:
# Run the agent tests
# For each test case, call the agent and collect the response
# Store results for evaluation

print("=== Running Agent Tests ===")
test_results = []

for i, test_case in enumerate(test_cases):
    print(f"\nTest {i+1}: {test_case['id']}")
    print(f"Question: {test_case['question']}")
    print("-" * 50)
    
    try:
        # Call the agent
        response = ecohome_agent.invoke(
            question=test_case['question'],
            context=CONTEXT
        )
        
        # Store the result
        result = {
            'test_id': test_case['id'],
            'question': test_case['question'],
            'response': response,
            'expected_tools': test_case['expected_tools'],
            'expected_response': test_case['expected_response'],
            'timestamp': datetime.now().isoformat()
        }
        test_results.append(result)
                
    except Exception as e:
        print(f"Error: {e}")
        result = {
            'test_id': test_case['id'],
            'question': test_case['question'],
            'response': f"Error: {str(e)}",
            'expected_tools': test_case['expected_tools'],
            'expected_response': test_case['expected_response'],
            'timestamp': datetime.now().isoformat(),
            'error': str(e)
        }
        test_results.append(result)

print(f"\nCompleted {len(test_results)} tests")


=== Running Agent Tests ===

Test 1: ev_charging_1
Question: When should I charge my electric car tomorrow in San Francisco to minimize cost and maximize solar power?
--------------------------------------------------

Test 2: thermostat_1
Question: What thermostat settings should I use this week in San Francisco to save energy?
--------------------------------------------------

Test 3: appliance_schedule_1
Question: When is the best time to run my washing machine in San Francisco to reduce electricity costs?
--------------------------------------------------

Test 4: solar_max_1
Question: How can I maximize solar power usage over the next 3 days in San Francisco?
--------------------------------------------------

Test 5: cost_savings_1
Question: Estimate how much I can save in San Francisco by running my dishwasher during off-peak hours.
--------------------------------------------------

Test 6: ev_solar_combo_1
Question: Can I charge my EV using solar power this weekend in San Fra

## 4. Evaluate Responses

In [11]:
from agent import Agent
from pprint import pprint

# Initialize agent
advisor = Agent(instructions=ECOHOME_SYSTEM_PROMPT)

# Evaluation functions
def evaluate_final_response(response: str, expected_keywords: list) -> bool:
    response_lower = response.lower()
    return all(keyword.lower() in response_lower for keyword in expected_keywords)

def evaluate_tool_usage(response: list, expected_tools: list) -> bool:
    response_lower = response
    return all(tool.lower() in response_lower for tool in expected_tools)
def extract_final_response(response_dict: dict) -> str:
    """
    Extract the final AI-generated response from a response dict containing 'messages'.

    Args:
        response_dict (dict): The full response from the agent.

    Returns:
        str: The final response content.
    """
    messages = response_dict.get("messages", [])
    for msg in reversed(messages):
        if msg.__class__.__name__ == "AIMessage" and msg.content.strip():
            return msg.content.strip()
    return ""

def extract_response_and_tools(response_dict: dict) -> tuple[str, list[str]]:
    """
    Extract the final AI-generated response and list of tools called.

    Args:
        response_dict (dict): The full response from the agent.

    Returns:
        tuple: (final_response_text, list_of_tool_names)
    """
    tool_names = []
    final_response = ""
    for msg in response_dict.get("messages", []):
        if msg.__class__.__name__ == "AIMessage":
            # # Capture final response if present
            if msg.content.strip():
                final_response = msg.content.strip()

            # Extract tool calls
            tool_calls = msg.additional_kwargs.get("tool_calls", [])

            for call in tool_calls:
                #print(call,isinstance(call, dict) )
                tool_name = call.get("function", {}).get("name")
                if tool_name:
                    tool_names.append(tool_name)

    return final_response,tool_names
def evaluate_response(question: str, final_response: str, expected_response: str) -> dict:
    expected_keywords = [
        kw.strip().lower()
        for kw in expected_response.replace(".", "").replace(",", "").split()
        if len(kw.strip()) > 3
    ]
    matched_keywords = [kw for kw in expected_keywords if kw in final_response.lower()]
    return {
        "question": question,
        "expected_keywords": expected_keywords,
        "matched_keywords": matched_keywords,
        "match_ratio": round(len(matched_keywords) / len(expected_keywords), 2) if expected_keywords else 0.0,
        "passed": len(matched_keywords) >= max(1, len(expected_keywords) // 2),
        "final_response": final_response
    }


In [39]:
# 🧠 Evaluation functions

def evaluate_response(question, final_response, expected_response):
    expected_keywords = [kw.lower() for kw in expected_response.replace(".", "").split() if len(kw) > 3]
    response_lower = final_response.lower()

    def keyword_score(text, keywords):
        return sum(1 for kw in keywords if kw in text) / len(keywords)

    accuracy = round(keyword_score(response_lower, expected_keywords), 2)
    relevance = 1.0 if any(kw in response_lower for kw in expected_keywords[:3]) else 0.5
    completeness = round(len(set(expected_keywords).intersection(response_lower.split())) / len(expected_keywords), 2)
    usefulness = 1.0 if any(p in response_lower for p in ["recommend", "consider", "best option", "schedule"]) else 0.5

    feedback = []
    if accuracy < 0.8: feedback.append("Response missed key concepts.")
    if relevance < 0.8: feedback.append("Response could better align with user intent.")
    if completeness < 0.8: feedback.append("Response only partially addressed expected elements.")
    if usefulness < 0.8: feedback.append("Response lacks actionable guidance.")

    return {
        "accuracy": accuracy,
        "relevance": relevance,
        "completeness": completeness,
        "usefulness": usefulness,
        "overall_score": round((accuracy + relevance + completeness + usefulness) / 4, 2),
        "feedback": feedback
    }

def evaluate_tool_usage(messages_list, expected_tools):
    actual_tools = []
    for msg in messages_list:
        if msg.__class__.__name__ == "AIMessage":
            for call in msg.additional_kwargs.get("tool_calls", []):
                tool_name = call.get("function", {}).get("name")
                if tool_name:
                    actual_tools.append(tool_name)

    appropriate = all(tool in expected_tools for tool in actual_tools)
    complete = all(tool in actual_tools for tool in expected_tools)

    feedback = []
    if not appropriate: feedback.append("Some tools used were not expected.")
    if not complete: feedback.append("Not all required tools were used.")

    return {
        "actual_tools": actual_tools,
        "tool_appropriateness": appropriate,
        "tool_completeness": complete,
        "tool_score": round((appropriate + complete) / 2, 2),
        "feedback": feedback
    }

def generate_evaluation_report(results):
    total = len(results)
    avg_scores = {
        "accuracy": round(sum(r["response"]["accuracy"] for r in results) / total, 2),
        "relevance": round(sum(r["response"]["relevance"] for r in results) / total, 2),
        "completeness": round(sum(r["response"]["completeness"] for r in results) / total, 2),
        "usefulness": round(sum(r["response"]["usefulness"] for r in results) / total, 2),
        "tool_score": round(sum(r["tools"]["tool_score"] for r in results) / total, 2),
    }

    weaknesses = []
    for r in results:
        weaknesses.extend(r["response"]["feedback"] + r["tools"]["feedback"])

    recommendations = []
    if avg_scores["accuracy"] < 0.8:
        recommendations.append("Improve factual grounding and keyword coverage.")
    if avg_scores["completeness"] < 0.8:
        recommendations.append("Ensure all expected elements are addressed.")
    if avg_scores["tool_score"] < 1.0:
        recommendations.append("Refine tool selection logic for completeness and relevance.")

    return {
        "average_scores": avg_scores,
        "total_cases": total,
        "recommendations": list(set(recommendations)),
        "common_issues": list(set(weaknesses))
    }

def display_evaluation_report(report):
    print("\n📊 Evaluation Report")
    print("-" * 40)
    print(f"Total Test Cases Evaluated: {report['total_cases']}")
    print("\n🔢 Average Scores:")
    for metric, score in report["average_scores"].items():
        print(f"  {metric.capitalize():<15}: {score:.2f}")
    print("\n⚠️ Common Issues:")
    for issue in report["common_issues"]:
        print(f"  - {issue}")
    print("\n💡 Recommendations:")
    for rec in report["recommendations"]:
        print(f"  - {rec}")

In [40]:
evaluation_results = []
# Run evaluation
for i, test in enumerate(test_cases):
    print(f"\n--- Test Case: {test['id']} ---")
    response = test_results[i]
    #final_response = extract_final_response(response["response"])
    final_response,final_tools = extract_response_and_tools(test_results[i]["response"])
    result = evaluate_response(test["question"], final_response, test["expected_response"])
    tool_check = evaluate_tool_usage(final_tools, test["expected_tools"])
    evaluation_results.append({
        "id": test["id"],
        "response": result,
        "tools": tool_check
    })




--- Test Case: ev_charging_1 ---

--- Test Case: thermostat_1 ---

--- Test Case: appliance_schedule_1 ---

--- Test Case: solar_max_1 ---

--- Test Case: cost_savings_1 ---

--- Test Case: ev_solar_combo_1 ---

--- Test Case: hvac_seasonal_1 ---

--- Test Case: battery_opt_1 ---

--- Test Case: smart_appliance_1 ---

--- Test Case: solar_roi_1 ---

📊 Evaluation Report
----------------------------------------
Total Test Cases Evaluated: 10

🔢 Average Scores:
  Accuracy       : 0.39
  Relevance      : 0.80
  Completeness   : 0.27
  Usefulness     : 0.95
  Tool_score     : 0.50

⚠️ Common Issues:
  - Response only partially addressed expected elements.
  - Response lacks actionable guidance.
  - Not all required tools were used.
  - Response missed key concepts.
  - Response could better align with user intent.

💡 Recommendations:
  - Ensure all expected elements are addressed.
  - Refine tool selection logic for completeness and relevance.
  - Improve factual grounding and keyword cove

In [41]:
report = generate_evaluation_report(evaluation_results)


In [42]:
display_evaluation_report(report)


📊 Evaluation Report
----------------------------------------
Total Test Cases Evaluated: 10

🔢 Average Scores:
  Accuracy       : 0.39
  Relevance      : 0.80
  Completeness   : 0.27
  Usefulness     : 0.95
  Tool_score     : 0.50

⚠️ Common Issues:
  - Response only partially addressed expected elements.
  - Response lacks actionable guidance.
  - Not all required tools were used.
  - Response missed key concepts.
  - Response could better align with user intent.

💡 Recommendations:
  - Ensure all expected elements are addressed.
  - Refine tool selection logic for completeness and relevance.
  - Improve factual grounding and keyword coverage.


In [ ]:
https://github.com/Surveshchauhan/Energy-Advisor.git